In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install opencv-python

In [4]:
#downloading all the required libraraies
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import glob# for data extraction from xml file
from xml.etree import ElementTree

In [5]:
annotations_directory = '../input/face-mask-detection/annotations'
images_directory = '../input/face-mask-detection/images'

In [6]:
annotation_files = !ls '../input/face-mask-detection/annotations'
annotation_files[:10]

In [7]:
images_files = !ls '../input/face-mask-detection/images'
images_files[:10]

In [8]:
len(annotation_files), len(images_files)

In [9]:
import xml.etree.ElementTree as ET
dataset = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "label":[],    
            "file":[],
            "width":[],
            "height":[],
           }

for anno in glob.glob(annotations_directory+"/*.xml"):
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in elem.tag:
            for attr in list(elem):
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['label']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]    

In [10]:
df=pd.DataFrame(dataset)
df.head()
#print(df['name'].unique())

In [11]:
df['annotation_file'] = df['file'] + '.xml' #added one new column for xml file
df['image_file'] = df['file'] + '.png'#added one new column for img file

In [12]:
#df
#this will count number of people in the image maksssksksss737 with visible faces
sum=0;
for i in df['file']:
    if(i=='maksssksksss737'):
        sum=sum+1
print(sum)

In [13]:
import cv2
def show_image(image):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.show()
def convert_to_RGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
def crop_image(image):
        x = df['xmin'].iloc[i]
        y = df['ymin'].iloc[i]
        width = df['xmax'].iloc[i]
        height = df['ymax'].iloc[i]
        cropped_image = image[y:height, x:width]
    #return cropped_image


***The beauty of this adatset which makes it different from others is that it contain images with several people and not just one due to which a separate xml file is used which describes every person visible face and hence we need to crop all the people:(***

In [14]:
def crop_image(image):
    for i in df['file']:
        x = annotations_info_df['xmin'].iloc[i]
        y = annotations_info_df['ymin'].iloc[i]
        width = annotations_info_df['xmax'].iloc[i]
        height = annotations_info_df['ymax'].iloc[i]
        cropped_image = image[y:height, x:width]
    return cropped_image

In [15]:
!ls '../input/face-mask-detection'

In [16]:
directory = 'cropped_images'
parent_directory = '/kaggle/working'
path = os.path.join(parent_directory, directory)
os.mkdir(path)


In [17]:
!ls './'

In [18]:
df['cropped_image_file'] = df['file']
df

In [19]:
for i in range(len(df['file'])):
    image_filepath = '../input/face-mask-detection/images/' + df['image_file'].iloc[i]
    image = cv2.imread(image_filepath)
    df['cropped_image_file'].iloc[i] = df['cropped_image_file'].iloc[i] + '-' + str(i) + '.png'
    cropped_image_filename = df['cropped_image_file'].iloc[i]
    xmin = df['xmin'].iloc[i]
    ymin = df['ymin'].iloc[i]
    xmax = df['xmax'].iloc[i]
    ymax = df['ymax'].iloc[i]

    # Crop The Image Based on The Values Above
    cropped_image = image[ymin:ymax, xmin:xmax]
    
    # Save Cropped Image
    cropped_image_directory = os.path.join('./cropped_images', cropped_image_filename) 
    cv2.imwrite(cropped_image_directory, cropped_image)

In [20]:
df

In [21]:
cropped_images_files = !ls './cropped_images'
cropped_images_files[:10]

In [22]:
#df.shuffle()
test_df = df[:1200]
train_df = df[1200:]


In [23]:
classes = list(train_df['label'].unique())
classes

Now let's have some check about the labels of an image...

In [24]:
train_df[train_df['file'] == 'maksssksksss139']['label'].unique()

In [25]:
image_139_path = '../input/face-mask-detection/images/maksssksksss139.png'
image_139 = cv2.imread(image_139_path)
image_139

In [26]:
image_139_rgb = convert_to_RGB(image_139)
show_image(image_139_rgb)

In [27]:
image_139_df = train_df[train_df['file'] == 'maksssksksss139']
image_139_df

In [28]:
with_mask_list, without_mask_list, incorrectly_worn_list = [], [], []
for i in range(len(image_139_df)):
    bounding_box = [image_139_df['xmin'].iloc[i], image_139_df['ymin'].iloc[i],
                    image_139_df['xmax'].iloc[i], image_139_df['ymax'].iloc[i]]
    if image_139_df['label'].iloc[i] == 'with_mask':
        with_mask_list.append(bounding_box)
    elif image_139_df['label'].iloc[i] == 'without_mask':
        without_mask_list.append(bounding_box)
    else:
        incorrectly_worn_list.append(bounding_box)
        
found_objects_dict = {'With Mask': with_mask_list, 
                      'Without Mask': without_mask_list, 
                      'Incorrectly Worn': incorrectly_worn_list}
found_objects_dict

Now let's check whether the images are correctly albelled or not directly on the image itself using openCV

In [29]:
for key, value in found_objects_dict.items():
    for i in range(len(value)):
        color = (0, 255, 0) # green
        text = 'Mask'
        if key == 'Without Mask':
            color = (255, 0, 0) # red
            text = 'No Mask'
        elif key == 'Incorrectly Worn':
            color = (255, 255, 0) # yellow
            text = 'Incorrect'
        start_point = (value[i][0], value[i][1])
        end_point = (value[i][2], value[i][3])
        cv2.rectangle(image_139_rgb, start_point, end_point, color = color, thickness = 2)
        cv2.putText(image_139_rgb, org = (value[i][0] - 8, value[i][1] - 3), text = text, 
                    fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.5, color = color)

In [30]:
show_image(image_139_rgb)

In [31]:
train_df['label'].value_counts()

In [32]:
sorted_label_df = pd.DataFrame(train_df['label'].value_counts()).reset_index()
sorted_label_df.rename(columns = {'index': 'label', 'label': 'count'}, inplace = True)
sorted_label_df

Now let's have a look in distribution of the people with three classes

In [33]:
plt.style.use('seaborn')
plt.figure(figsize = (8, 6))
barplot = sns.barplot(x = 'count', y = 'label', data = sorted_label_df, orient = 'horizontal', 
                      palette = ['green', 'red', 'yellow'])
plt.title('Distribution of Labels', fontsize = 20, fontweight = 'bold')
plt.xlabel('Count', fontsize = 15, fontweight = 'bold')
plt.ylabel('Label', fontsize = 15, fontweight = 'bold')

for p in barplot.patches:
    width = p.get_width() * 1.0
    print(width)
    print(sum(sorted_label_df['count']), 2)
    percentage = (width * 100 / (sum(sorted_label_df['count']), 2))
    plt.text(x = width + 15, y = p.get_y() + 0.55 * p.get_height(), s = f'{int(width)}\n({percentage} %)')

plt.show()

Now we will do image preprocessing

In [34]:
cropped_image_path = './cropped_images/' + train_df['cropped_image_file'].iloc[0]
cropped_image = cv2.imread(cropped_image_path)
cropped_image.shape

In [35]:
image_width = []
image_height = []
for i in range(len(train_df)):
    cropped_image_path = './cropped_images/' + train_df['cropped_image_file'].iloc[i]
    cropped_image = cv2.imread(cropped_image_path)
    image_width.append(cropped_image.shape[0])
    image_height.append(cropped_image.shape[1])

In [36]:
sns.histplot(image_width, kde = True)
plt.title('Image Width Distribution', fontsize = 16, fontweight = 'bold')
plt.xlabel('Image Width', fontweight = 'bold')
plt.ylabel('Count', fontweight = 'bold')
plt.show()

In [37]:
sns.histplot(image_height, kde = True)
plt.title('Image Height Distribution', fontsize = 16, fontweight = 'bold')
plt.xlabel('Image Height', fontweight = 'bold')
plt.ylabel('Count', fontweight = 'bold')
plt.show()

In [38]:
image_target_size = (int(np.median(image_width)), int(np.median(image_height)))
image_target_size

In [39]:
from keras_preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale = 1. / 255., validation_split = 0.25)

train_generator = train_image_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = './cropped_images',
    x_col = 'cropped_image_file',
    y_col = 'label',
    subset = 'training',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
    )
valid_generator = train_image_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = './cropped_images',
    x_col = 'cropped_image_file',
    y_col = 'label',
    subset = 'validation',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

In [40]:
test_image_generator = ImageDataGenerator(rescale = 1. / 255.)

test_generator = train_image_generator.flow_from_dataframe(
    dataframe = test_df,
    directory = './cropped_images',
    x_col = 'cropped_image_file',
    y_col = 'label',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

In [41]:
print(train_generator)
print(valid_generator)
print(test_generator)

In [42]:
input_shape = [int(np.median(image_width)), int(np.median(image_height)), 3]

Now the modelling will take place

In [89]:
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu",input_shape=input_shape),
    keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=input_shape),
    keras.layers.MaxPool2D(pool_size=3,padding='same'),
    keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu",input_shape=input_shape),
    keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=input_shape),
    keras.layers.MaxPool2D(pool_size=3,padding='same'),
    keras.layers.Flatten(),
    keras.layers.Dense(units = len(classes), activation = 'softmax')
])

In [103]:
model.compile(loss = 'categorical_crossentropy',
                optimizer = keras.optimizers.Adam(),
                metrics = ['accuracy', keras.metrics.Recall()])

history_1 = model.fit(train_generator, epochs = 5, steps_per_epoch = len(train_generator), 
                        validation_data = valid_generator, validation_steps = len(valid_generator))

In [104]:
val_loss,val_acc=model.evaluate(train_generator)

val_loss,val_acc=model.evaluate(test_generator)

In [105]:
val_loss,val_acc=model.evaluate(valid_generator)

In [106]:
val_loss,val_acc=model.evaluate(test_generator)

In [86]:
result=pd.DataFrame(history_1.history)
result

In [87]:
result.plot()